In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = (8.0, 10.0)

In [ ]:
import os

homeDir = os.environ["HOME"]
dataDir = f'{homeDir}/Data/coco2017'
imageDir = f'{dataDir}/val2017'
annFile = f'{dataDir}/annotations/instances_val2017.json'
print(annFile)

In [ ]:
# initialize COCO api for instance annotations
coco=COCO(annotation_file=annFile)

In [ ]:
# display COCO categories and supercategories
cats = coco.loadCats(coco.getCatIds())
id_to_cat = {cat['id']: cat for cat in cats}
print(id_to_cat)

In [ ]:
# get all images containing given categories, select one at random
catIds = coco.getCatIds(catNms=['person','dog','skateboard']);
imgIds = coco.getImgIds(catIds=catIds );
imgIds = coco.getImgIds(imgIds = [324158])
img = coco.loadImgs(imgIds[np.random.randint(0,len(imgIds))])[0]
print(img)

In [ ]:
# load and display image
# I = io.imread('%s/images/%s/%s'%(dataDir,dataType,img['file_name']))
# use url to load image
imgFile = img['file_name']
I = io.imread(f'{imageDir}/{imgFile}')
plt.axis('off')
plt.imshow(I)
plt.show()

In [ ]:
# load and display instance annotations
plt.imshow(I); plt.axis('off')
annIds = coco.getAnnIds(imgIds=img['id'], catIds=catIds, iscrowd=None)
anns = coco.loadAnns(annIds)
coco.showAnns(anns)

In [ ]:
# initialize COCO api for person keypoints annotations
annFile = f'{dataDir}/annotations/person_keypoints_val2017.json'
coco_kps=COCO(annFile)

In [ ]:
# load and display keypoints annotations
plt.imshow(I); plt.axis('off')
ax = plt.gca()
annIds = coco_kps.getAnnIds(imgIds=img['id'], catIds=catIds, iscrowd=None)
anns = coco_kps.loadAnns(annIds)
coco_kps.showAnns(anns)

In [ ]:
# initialize COCO api for caption annotations
annFile = f'{dataDir}/annotations/captions_train2017.json'
coco_caps=COCO(annFile)

In [ ]:
# load and display caption annotations
annIds = coco_caps.getAnnIds(imgIds=img['id']);
anns = coco_caps.loadAnns(annIds)
coco_caps.showAnns(anns)
plt.imshow(I); plt.axis('off'); plt.show()